In [1]:
from google.colab import drive #подключение гугл диска
drive.mount ('/content/drive/')
%cd /content/drive/MyDrive

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive


In [21]:
import pandas as pd # Импортировал библиотеку pandas
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt # Импортировал библиотеку pyplot
import numpy as np # Импортировал библиотеку numpy


from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PowerTransformer, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import GridSearchCV
import xgboost as xgb



# Загрузим данные из файла
data = pd.read_csv('/content/drive/MyDrive/Laptop_price.csv')

# Выберем столбец Price в качестве целевого
y = data['Price']

# Удалим столбец Price из X
X = data.drop('Price', axis=1)

# Сделаем сплит
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
num_columns = list(X_train.select_dtypes(exclude=['object']))
num_columns

['Processor_Speed', 'RAM_Size', 'Storage_Capacity', 'Screen_Size', 'Weight']

In [4]:
cat_columns = list(X_train.select_dtypes(include=['object']))
cat_columns

['Brand']

In [11]:
numerical = Pipeline(steps=[
    ("simple_imputer", SimpleImputer()), #подбор стратегии по заполнению пропущенных значений
    ("power_transform", PowerTransformer()), #приведение данных к нормальному распределению
    ('Scaler', StandardScaler())#стандартизирует данные
])

In [12]:
categorical = Pipeline(steps=[
    ("simple_imputer", SimpleImputer(strategy='')),
    ("OneHotEncoder", OneHotEncoder(handle_unknown='ignore', # drop = ['first', 'if_binary']
                                    sparse_output=False))#

    ])

In [7]:
ct = ColumnTransformer([
    ("numerical", numerical, num_columns),
    ("categorical", categorical, cat_columns)
])

In [8]:
pipe = Pipeline(steps=[
    ("ct", ct),
    ("XGBRegressor", XGBRegressor(booster=None, callbacks=None,
                                  gamma=None, grow_policy=None,
                                  importance_type=None, interaction_constraints=None, learning_rate=None,
                                  max_bin=None, max_cat_threshold=None, random_state=None))
])

In [23]:
param_grid={
    'XGBRegressor__n_estimators':[50],
    'XGBRegressor__learning_rate':[0,1],
    'ct__numerical': [StandardScaler(),RobustScaler()]# в пайплайне есть инструмент позволяющий оптимизировать подбор нужных гиперпараметров.
    #GridSearch - это очень мощный инструмент для автоматического подбирания параметров для моделей машинного обучения.
    #GridSearchCV находит наилучшие параметры, путем обычного перебора: он создает модель для каждой возможной комбинации параметров.
    #Важно отметить, что такой подход может быть весьма времязатратным

}

In [22]:
clf = GridSearchCV(pipe, param_grid, verbose=3)
clf.fit(X_train, y_train)

clf.score(X_test, y_test)

model=clf.best_estimator_
clf.best_params_

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5] END XGBRegressor__learning_rate=0, XGBRegressor__n_estimators=60, ct__numerical=StandardScaler();, score=nan total time=   0.0s
[CV 2/5] END XGBRegressor__learning_rate=0, XGBRegressor__n_estimators=60, ct__numerical=StandardScaler();, score=nan total time=   0.0s
[CV 3/5] END XGBRegressor__learning_rate=0, XGBRegressor__n_estimators=60, ct__numerical=StandardScaler();, score=nan total time=   0.0s
[CV 4/5] END XGBRegressor__learning_rate=0, XGBRegressor__n_estimators=60, ct__numerical=StandardScaler();, score=nan total time=   0.0s
[CV 5/5] END XGBRegressor__learning_rate=0, XGBRegressor__n_estimators=60, ct__numerical=StandardScaler();, score=nan total time=   0.0s
[CV 1/5] END XGBRegressor__learning_rate=0, XGBRegressor__n_estimators=60, ct__numerical=RobustScaler();, score=nan total time=   0.0s
[CV 2/5] END XGBRegressor__learning_rate=0, XGBRegressor__n_estimators=60, ct__numerical=RobustScaler();, score=nan tota

ValueError: 
All the 20 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 401, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 359, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "/usr/local/lib/python3.10/dist-packages/joblib/memory.py", line 312, in __call__
    return self.func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 893,